In [145]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt

import aot


In [146]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(
    open(stimuli_temp_path), Loader=yaml.FullLoader)

settings_root_path = base_dir / core_settings["paths"]["settings_path"]
video_db_path = base_dir / 'data/videos/database_originals.tsv'
video_db = pd.read_csv(video_db_path, sep='\t')
main_settings_path = settings_root_path / 'main'

videos_names = []
for i in range(1,len(video_db)):
    if video_db['grade'][i] == 'j' or video_db['grade'][i] == 'NA':
        videos_names.append(video_db['video_name'][i])
print("videos number: ", len(videos_names))

origin_videos_number = len(videos_names)
subject_number = core_settings['various']['subject_number']
session_number = core_settings['various']['session_number']
run_number = core_settings['various']['run_number']
# only for main run
blank_number_per_run = core_settings['various']['blank_number_per_run']
unique_video_number_per_session = core_settings['various']['unique_video_number_per_session']
video_number_per_run_main = int(
    (unique_video_number_per_session*4) / run_number)
# only 1 subject and 1 session for grading
video_number_per_run_grading = int(origin_videos_number / run_number)


videos number:  1405


In [147]:
def read_settings_file(subject, ses, run):
    settings_file = main_settings_path / f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(ses).zfill(2)}_run_{str(run).zfill(2)}.yml"
    settings = yaml.load(open(settings_file), Loader=yaml.FullLoader)
    return settings

In [148]:
def read_settings_files_session(subject, ses):#return a list of settings files for a given session
    settings_files = [read_settings_file(subject, ses, run) for run in range(1,run_number+1)]
    sessions_movies = [settings['stimuli']['movie_files'] for settings in settings_files]
    return sessions_movies

In [149]:
def check_video_replay_of_session(session_movies):
    #check if the same video is replayed in a session
    #append all the lists of movies in a session
    all_movies = []
    for run_movies in session_movies:
        all_movies += run_movies
    #remove the blanks
    all_movies = [movie for movie in all_movies if movie != 'blank']
    #check if there is a replay
    stack = []
    for movie in all_movies:
        if movie in stack:
            stack.remove(movie)
        else:
            stack.append(movie)
    return len(stack) == 0

def check_video_replay_all_session():
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            session_movies = read_settings_files_session(subject, ses)
            #print(f"subject {subject}, session {ses} movies: {session_movies}")
            is_replay = check_video_replay_of_session(session_movies)
            print(f"subject {subject}, session {ses} replay: {is_replay}")

check_video_replay_all_session()

subject 1, session 1 replay: True
subject 1, session 2 replay: True
subject 1, session 3 replay: True
subject 1, session 4 replay: True
subject 1, session 5 replay: True
subject 1, session 6 replay: True
subject 2, session 1 replay: True
subject 2, session 2 replay: True
subject 2, session 3 replay: True
subject 2, session 4 replay: True
subject 2, session 5 replay: True
subject 2, session 6 replay: True
subject 3, session 1 replay: True
subject 3, session 2 replay: True
subject 3, session 3 replay: True
subject 3, session 4 replay: True
subject 3, session 5 replay: True
subject 3, session 6 replay: True
subject 4, session 1 replay: True
subject 4, session 2 replay: True
subject 4, session 3 replay: True
subject 4, session 4 replay: True
subject 4, session 5 replay: True
subject 4, session 6 replay: True
subject 5, session 1 replay: True
subject 5, session 2 replay: True
subject 5, session 3 replay: True
subject 5, session 4 replay: True
subject 5, session 5 replay: True
subject 5, ses

In [150]:
def check_video_direction_duality_of_session(session_movies):
    #check whether the videos in a session are duality pairs(R_S_movie,S_movie)
    #append all the lists of movies in a session
    def check_duality(video1,video2):
        if video1.startswith('R_') and video2.startswith('S_'):
            return video1[4:] == video2[2:]
        elif video1.startswith('S_') and video2.startswith('R_'):
            return video1[2:] == video2[4:]
        else:
            return False
    def is_duality_in_stack(video, video_stack):#with side effect that remove the duality pair
        for v in video_stack:
            if check_duality(video,v):
                video_stack.remove(v)
                return True
        return False
    all_movies = []
    for run_movies in session_movies:
        all_movies += run_movies
    #remove the blanks
    all_movies = [movie for movie in all_movies if movie != 'blank']
    stack = []
    for movie in all_movies:
        if not is_duality_in_stack(movie, stack):
            stack.append(movie)
    return len(stack) == 0

def check_video_direction_duality_all_session():
    for subject in range(1, subject_number+1):
        for ses in range(1, session_number+1):
            session_movies = read_settings_files_session(subject, ses)
            #print(f"subject {subject}, session {ses} movies: {session_movies}")
            is_duality = check_video_direction_duality_of_session(session_movies)
            print(f"subject {subject}, session {ses} duality: {is_duality}")

check_video_direction_duality_all_session()
    

subject 1, session 1 duality: True
subject 1, session 2 duality: True
subject 1, session 3 duality: True
subject 1, session 4 duality: True
subject 1, session 5 duality: True
subject 1, session 6 duality: True
subject 2, session 1 duality: True
subject 2, session 2 duality: True
subject 2, session 3 duality: True
subject 2, session 4 duality: True
subject 2, session 5 duality: True
subject 2, session 6 duality: True
subject 3, session 1 duality: True
subject 3, session 2 duality: True
subject 3, session 3 duality: True
subject 3, session 4 duality: True
subject 3, session 5 duality: True
subject 3, session 6 duality: True
subject 4, session 1 duality: True
subject 4, session 2 duality: True
subject 4, session 3 duality: True
subject 4, session 4 duality: True
subject 4, session 5 duality: True
subject 4, session 6 duality: True
subject 5, session 1 duality: True
subject 5, session 2 duality: True
subject 5, session 3 duality: True
subject 5, session 4 duality: True
subject 5, session 5